In [5]:
!pip install beautifulsoup4
!pip install selenium
!pip install tqdm

You should consider upgrading via the 'c:\users\sungwoo\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


^C


You should consider upgrading via the 'c:\users\sungwoo\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [1]:
# 시스템
import os
import sys
import re
# 크롤링
import urllib.request
import requests
import json
from scrapy.http import TextResponse
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm

import pandas as pd

C:\Users\sungwoo\AppData\Local\Programs\Python\Python38-32\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# 태그 데이터를 삭제하는 코드
def tag_remove(word):

    word = re.sub('(<([^>]+)>)', '', word)

    return word

# 한글만 남기는 코드 -> 태그만 삭제해야함
def preprocess_sentence_kr(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
    w = w.strip()
    return w

# 한글 영어만 남기는 코드 -> 태그만 삭제해야함
def preprocess_sentence(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣a-zA-Z?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
    w = w.strip()
    return w

In [3]:
# url을 받아와서 글을 파싱해오는 코드

def blog_content_parsing(url):
    blog_req = requests.get(url)
    blog_bs = BeautifulSoup(blog_req.text, "html.parser")
    
    result = []
    
    # 글 내용 크롤링
    for i in blog_bs.select(".se-text"):
        for j in i.select(".se-module-text > p"):
            txt = j.text
            if txt != "\u200b":
                result.append(preprocess_sentence(txt))
    content = " ".join(result)

    # 이미지 내용 크롤링
    
    img_craw = blog_bs.select(".se-image")
    first_img = img_craw[0].select_one(".se-module-image-link > img")["data-lazy-src"]
    last_img = img_craw[len(img_craw)-1].select_one(".se-module-image-link > img")["data-lazy-src"]
                
    return content, first_img, last_img

In [81]:
def blog_content_parsing2(url):
    blog_req = requests.get(url)
    blog_bs = BeautifulSoup(blog_req.text, "html.parser")
    
    result = []
    content, first_img, last_img = "", "" ,""
    try:
        # 현재 여기서 안되는것은 스마트에디터 3로 작성된 경우이다.
        # 이미지 내용 크롤링
        img_craw = blog_bs.select(".se-image")
        first_img = img_craw[0].select_one(".se-module-image-link > img")["data-lazy-src"]
        last_img = img_craw[len(img_craw)-1].select_one(".se-module-image-link > img")["data-lazy-src"]
        # 글 내용 크롤링
        for i in blog_bs.select(".se-text"):
            for j in i.select(".se-module-text > p"):
                txt = j.text
                if txt != "\u200b":
                    result.append(txt)
        content = " ".join(result)
    except:
        return False, content, first_img, last_img
        
    return True, content, first_img, last_img

In [85]:
# 스마트 에디터 1 or 2
url = "https://blog.naver.com/PostView.naver?blogId=rladnjsxo64&logNo=222571874963"
url2 = "https://blog.naver.com/PostView.naver?blogId=playgeo&logNo=222345388278&redirect=Dlog&widgetTypeCall=true&topReferer=https%3A%2F%2Fsearch.naver.com%2Fsearch.naver%3Fquery%3D%25E2%2599%25A1%26nso%3D%26where%3Dblog%26sm%3Dtab_opt&directAccess=false"

# 스마트에디터 3
url3 = "https://blog.naver.com/PostView.naver?blogId=qkr200135&logNo=222492039405&redirect=Dlog&widgetTypeCall=true&topReferer=https%3A%2F%2Fsearch.naver.com%2Fsearch.naver%3Fquery%3D%25E2%2599%25A1%26nso%3D%26where%3Dblog%26sm%3Dtab_opt&directAccess=false"


In [86]:
#SE-90837e56-b162-11eb-bb27-7bed83a631bf > div > div > div > a > img

# 스마트에디터3는 일단 나중에 처리 진행
blog_content_parsing2(url3)

(False, '', '', '')

In [11]:
# blog가 네이버 블로그인지 아닌지 판별하는 코드

def naver_blog_check(code):
    
    if code[0].isdigit():
        return False
    else:
        return True


# 모든 단어를 하나씩 나누어서 확인하자.
# 

def naver_title_check(search_word, title):
     
    # search_word = preprocess_sentence_kr(search_word)
    
    for words in search_word.split():
        for word in words:
            if word.lower() not in title.lower():
                return True



# 검색 결과가 title에 들어가있다면 해당 내용을 없앤다.
# 아니면 해당 내용은 다른 내용이라고 판단한다.

# 띄어쓰기를 안하고 -로 나눈경우
# 

# def naver_title_check(search_word, title):
#     # -로 제품이 안나뉜 경우 -> 소문자로만 판단
     
#     total_cnt = len(search_word.split())
#     cnt = 0
    
#     for word in search_word.split():
#         word = word.lower()
#         if word in title.lower(): # 대소문자 구분 x
#             cnt += 1
#     if total_cnt == cnt:
#         return False
#     else:
#         return True 


# # 글자 수 중 반 이상이 title에 들어가있다면 패스 아니면 해당 내용은 다른 내용이라고 판단한다.

# def naver_title_check(search_word, title):
#     total_cnt = len(title.split())
#     cnt = 0
    
#     for word in search_word.split():
        
#         if word in title:
#             cnt += 1
            
#     if total_cnt// 2 <= cnt:
#         return False
#     else:
#         return True 
        
        

In [15]:
# 이름이 들어오면 해당 내용을 이용하여 블로그를 파싱한다.

# search_word : 어떤 제품을 검색하는지
# repeat_num : 해당 제품이 제목에서 몇번 안나올때까지 검색을 진행할 지
def item_parsing(search_word, start, display, repeat_num):
   
    blog_search = naver_api(search_word, start, display)
    print("start ++++++++++++++++++++")
    # result : url
    # writer : 작성자 이름
    # code : 블로그 코드
    # title : 블로그 제목
    # description : 블로그 요약 설명
    blog_result = []
    blog_writer = []
    blog_code = []
    blog_title = []
    blog_description = []
    repeat = True
    cnt = 0
    # 제목에서 해당 제품의 이름이 5번 연속 안나온다면 패스
    while repeat:
        
        for blog in blog_search["items"]: 
             # 블로그 코드
            code = blog["link"][blog["link"].find("logNo=") + 6:]
            if naver_blog_check(code): # 네이버 블로그가 아닌경우 다음으로 넘기기
                continue

            # 이름
            writer = blog["bloggerlink"][23:]

            # 제목
            title = tag_remove(blog["title"])
            
            if naver_title_check(search_word, title):
                continue

            # 블로그 요약
            description = tag_remove(blog["description"])
            ## 데이터 추가하는 곳
            # 블로그 코드 추가
            blog_code.append(code)
            # 이름 추가
            blog_writer.append(writer)
            # 제목 추가
            blog_title.append(title)
            # 블로그 요약 추가
            blog_description.append(description)
            # 본문 크롤링을 위한 URL주소 추가
            blog_result.append(f"https://blog.naver.com/PostView.naver?blogId={writer}&logNo={code}")
        if start + display <= blog_search["total"]:
            start += display
            blog_search = naver_api(search_word, start, display)
            print(f"start = {start}")
            print("repeat start ++++++++++++++++++++")
        else:
            print("repeat End")
            repeat = False
    return blog_result, blog_writer, blog_code, blog_title, blog_description
        


# # 이름이 들어오면 해당 내용을 이용하여 블로그를 파싱한다.

# # search_word : 어떤 제품을 검색하는지
# # repeat_num : 해당 제품이 제목에서 몇번 안나올때까지 검색을 진행할 지
# def item_parsing(search_word, start, display, repeat_num):

    
#     blog_search = naver_api(search_word, start, display)
#     print("start ++++++++++++++++++++")
    
#     # result : url
#     # writer : 작성자 이름
#     # code : 블로그 코드
#     # title : 블로그 제목
#     # description : 블로그 요약 설명
#     blog_result = []
#     blog_writer = []
#     blog_code = []
#     blog_title = []
#     blog_description = []
    
#     repeat = True
#     cnt = 0
#     # 제목에서 해당 제품의 이름이 5번 연속 안나온다면 패스
#     while repeat:
        
#         if cnt == 3:
#             break
        
#         for blog in blog_search["items"]:
            
#              # 블로그 코드
#             code = blog["link"][blog["link"].find("logNo=") + 6:]
#             if naver_blog_check(code): # 네이버 블로그가 아닌경우 다음으로 넘기기
#                 continue
            
            
#             # 블로그 코드 추가
#             blog_code.append(code)
            
#             # 이름 추가
            
# #             print(f"cnt = {cnt}")
#             cnt += 1
#             # 이름
#             writer = blog["bloggerlink"][23:]
#             blog_writer.append(writer)


#             # 제목
#             title = tag_remove(blog["title"])
#             blog_title.append(title)
            
# #             if search_word not in title:
# #                 cnt += 1
# #             else:
# #                 cnt = 0
                
# #             if cnt == repeat_num:
# #                 repeat = False

#             # 블로그 요약
#             description = tag_remove(blog["description"])
#             blog_description.append(description)
            
#             blog_result.append(f"https://blog.naver.com/PostView.naver?blogId={writer}&logNo={code}")
        
#         if start + display <= blog_search["total"]:
#             start += display
#             blog_search = naver_api(search_word, start, display)
#             print(f"start = {start}")
#             print("repeat start ++++++++++++++++++++")
#         else:
#             print("repeat End")
#             repeat = False
            
#         cnt += 1
        
#     return blog_result, blog_writer, blog_code, blog_title, blog_description
        
    

In [12]:
def naver_api(search_word, start, display):
    client_id = "GvNa2sBgFDA6v7ujnaz0"
    client_secret = "Yo0jOskXlZ"
    
    encText = urllib.parse.quote(search_word)
    
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
    url += f"&start={start}"
    url += f"&display={display}"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
    else:
        print("Error Code:" + rescode)
    return json.loads(response_body.decode('utf-8'))

In [13]:
t = naver_api("wf-1000xm4 후기", 1, 30)
t

{'lastBuildDate': 'Tue, 19 Apr 2022 10:29:11 +0900',
 'total': 355,
 'start': 1,
 'display': 30,
 'items': [{'title': '무선 이어폰 종결 끝판왕 소니 <b>WF-1000XM4</b> 샀습니다',
   'link': 'https://blog.naver.com/khjkhj2804?Redirect=Log&logNo=222645074782',
   'description': '<b>WF-1000XM4</b> 가 현재 끝판왕인 이유 일단, 가격 깡패 &amp; 음질 깡패 최고임 이제 한 달 넘게 사용하고 쓰는 <b>후기</b> 무엇보다 고성능 라인업들 가격대비 가격 앞 자리 단위 자체가 내려가 있습니다. 가격 자체가 깡패인데... ',
   'bloggername': '겜잘알 갓플머',
   'bloggerlink': 'https://blog.naver.com/khjkhj2804',
   'postdate': '20220211'},
  {'title': '소니 <b>WF-1000XM4</b> 노이즈 캔슬링 이어폰 개봉 <b>후기</b>... ',
   'link': 'https://blog.naver.com/rladnjsxo64?Redirect=Log&logNo=222571874963',
   'description': '지난 몇 주간 소니 <b>WF-1000XM4</b>에 대한 사용 <b>후기</b>를 남겼는데 개봉기를 빼놓을 수 없죠! 오늘은 조금(?) 늦은 개봉기를 작성해보겠습니다. <b>WF-1000XM4</b> 이어폰은 블랙과 실버 2가지 색상으로 출시를 했으며, 제가... ',
   'bloggername': '무라키의 데이 에세이',
   'bloggerlink': 'https://blog.naver.com/rladnjsxo64',
   'postdate': '20211118'},
  {'title': '무선 노이즈 캔슬링 이어폰 소니 <b>WF-1000XM4</b

In [16]:
test = item_parsing("wf-1000xm4 후기", 1, 100, 3)

start ++++++++++++++++++++
start = 101
repeat start ++++++++++++++++++++
start = 201
repeat start ++++++++++++++++++++
start = 301
repeat start ++++++++++++++++++++
repeat End


In [5]:
def search_word(word):
    result = []
    append_data = ["후기", "리뷰"]
    for data in append_data:
        result.append(word + " " + data)
        
    return result

    

In [6]:
word = "랩씨 마카롱 스마트 커버"

In [7]:
search_word(word)

['랩씨 마카롱 스마트 커버 후기', '랩씨 마카롱 스마트 커버 리뷰']

In [19]:
a,b,c,d,e = test

In [20]:
a

['https://blog.naver.com/PostView.naver?blogId=rladnjsxo64&logNo=222571874963',
 'https://blog.naver.com/PostView.naver?blogId=feena74&logNo=222423856964',
 'https://blog.naver.com/PostView.naver?blogId=supapa13&logNo=222498158979',
 'https://blog.naver.com/PostView.naver?blogId=lifesdh&logNo=222414669555',
 'https://blog.naver.com/PostView.naver?blogId=xerotype2&logNo=222415306975',
 'https://blog.naver.com/PostView.naver?blogId=dongchimi212&logNo=222652612911',
 'https://blog.naver.com/PostView.naver?blogId=apricot_4&logNo=222658424853',
 'https://blog.naver.com/PostView.naver?blogId=skan5422&logNo=222583222026',
 'https://blog.naver.com/PostView.naver?blogId=semin1020&logNo=222661865508',
 'https://blog.naver.com/PostView.naver?blogId=rladnjsxo64&logNo=222565484664',
 'https://blog.naver.com/PostView.naver?blogId=ddihw&logNo=222452134060',
 'https://blog.naver.com/PostView.naver?blogId=pakseeun&logNo=222567999666',
 'https://blog.naver.com/PostView.naver?blogId=eter2000&logNo=222613

In [24]:
result = []
for i in tqdm(range(len(e))):
    result.append(blog_content_parsing(a[i]))

100%|██████████| 74/74 [00:19<00:00,  3.85it/s]


In [28]:
df = pd.DataFrame()
df["url"] = a
df["writer"] = b
df["title"] = d
df["description"] = e
df["content"] = result


In [29]:
df["협찬"] = 0
df["체험단"] = 0

df["협찬"][df["content"].str.contains("협찬")] = 1
df["체험단"][df["content"].str.contains("체험단")] = 1

C:\Users\sungwoo\AppData\Local\Temp\ipykernel_15272\3884111020.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["협찬"][df["content"].str.contains("협찬")] = 1
C:\Users\sungwoo\AppData\Local\Temp\ipykernel_15272\3884111020.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["체험단"][df["content"].str.contains("체험단")] = 1


In [56]:
df.loc[0, "content"].contains("소니")

AttributeError: 'str' object has no attribute 'contains'

In [49]:
df["content"]

0     지난 몇 주간 소니 1000 4에 대한 사용 후기를 남겼는데 개봉기를 빼놓을 수 없...
1     노이즈 캔슬링 이어폰의 왕의 귀환! 소니 1000 4가 발매가 되었습니다. 전작인 ...
2     드디어 나왔습니다! 크!! 기다리고 기다렸던 소니 1000 4 노이즈캔슬링 이어폰입...
3     소니에서 이번에 새로 정식 출시한 1000 4는 전작 1000 3에서 확연하게 달라...
4     안녕하세요, 밝은불꽃입니다! 요즘 날씨가 더워서 헤드폰 대신 블루투스 이어폰을 들고...
                            ...                        
91    중고 헤드폰 매입후기, 뱅앤올룹슨 안녕하세요. 판매 하시는 장비 및 전자기기를 빠르...
92    중고 헤드폰 매입후기, 애플 에어팟 맥스 무선 헤드폰 안녕하세요. 판매 하시는 장비...
93    최근 들어 가성비 좋은 제품으로 핫하게 떠오르고 있는 소니 블루투스 노이즈 캔슬링 ...
94    중고 무선 이어폰 매입후기, 애플 에어팟프로 안녕하세요. 판매 하시는 장비 및 전자...
95    무선 헤드폰 매입, 스컬캔디 블루투스 헤드폰 매입 후기 안녕하세요. 판매 하시는 장...
Name: content, Length: 96, dtype: object

In [32]:
df[df["content"].str.contains("coupa")]

,url,writer,title,description,content,협찬,체험단


In [74]:
df["coupang"] = 0

In [77]:
df["coupang"][df["content"].str.contains("coupa")] = 1

C:\Users\User\AppData\Local\Temp\ipykernel_10564\285214680.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["coupang"][df["content"].str.contains("coupa")] = 1


In [78]:
df[df["coupang"]==1]

,url,writer,title,description,content,협찬,체험단,coupang


In [105]:
cleanhtml("&nbsp coupa.ng")

'&nbspcoupa.ng'

In [36]:
df.to_csv("소닉_wh-1000xm42.csv", index= False,encoding="utf-8-sig")